## Enviorment

In [1]:
#verify user
!whoami
#verify user account, if running on google cloud, otherwise ignore
!gcloud auth list
#which enciorment/virtualenv running in
import sys
import os

print("Python running from:",sys.prefix)
print("Current working dir",os.getcwd())

desktop-2qv6e1h\flash
      Credentialed Accounts
ACTIVE  ACCOUNT
*       michael.zl.prime@gmail.com
Python running from: c:\program files\python36-64
Current working dir C:\Users\flash\Project\Yet-Another-Openpose-Implmentation



To set the active account, run:
    $ gcloud config set account `ACCOUNT`



In [2]:
#start tensor board
# must run 
#/usr/local/bin/tensorboard serve --logdir gs://dl_training_results/tensorboard --port 8889 --bind_all

---

In [3]:
import ipywidgets as widgets
import datetime
import tensorflow as tf

import dataset_functions
from models.six_stage_linear_model import ModelMaker
import callbacks
import dataset_builder
from config import *
import load_weights
import loss_metrics

print("Tensorflow version:",tf.version.VERSION)

Tensorflow version: 2.0.0


In [4]:
#override config
TPU_MODE=False

BATCH_SIZE=2
BASE_LEARNING_RATE=0.001
LEARNING_RATE_SCHEDUELE=np.zeros(1000)
LEARNING_RATE_SCHEDUELE[:3]=0.2
LEARNING_RATE_SCHEDUELE[3:20]=1
LEARNING_RATE_SCHEDUELE[20:40]=1
LEARNING_RATE_SCHEDUELE[40:100]=1
LEARNING_RATE_SCHEDUELE[100:]=0.3
LEARNING_RATE_SCHEDUELE*=BASE_LEARNING_RATE

In [5]:
if TPU_MODE:
    #import tpu_training.test_connectivity
    #tpu_training.test_connectivity.test_connectivity()
    print("Testing results bucket connectivity")
    !touch /tmp/test
    !gsutil cp /tmp/test {TENSORBOARD_PATH}/test
    !gsutil rm {TENSORBOARD_PATH}/test
    !gsutil cp /tmp/test {CHECKPOINTS_PATH}/test
    !gsutil rm {CHECKPOINTS_PATH}/test
    print("Testing dataset bucket connectivity")
    !gsutil ls gs://{GCS_TFRECORDS_BUCKETNAME} | head -4
    print("Testing TPU connectivity")
    !nmap -Pn -p8470 {TPU_IP}
    import tpu_training.init_tpu as init_tpu
    strategy,resolver=init_tpu.init_tpu()

In [6]:
#tf.config.experimental_connect_to_cluster(resolver)
#tpu_topo=tf.tpu.experimental.initialize_tpu_system(resolver)
#tf.keras.backend.clear_session()

---
# Make dataset

In [7]:
TF_parser = dataset_functions.TFrecordParser()
tfrecord_files_train,tfrecord_files_val=dataset_builder.get_tfrecord_filenames()
def build_training_ds(tfrecord_filenames:list)->tf.data.Dataset:
    """Generate training dataset from TFrecord file locations
    :param tfrecord_files should be list of correct TFrecord filename, either local or remote (gcs, with gs:// prefix)"""
    # TFrecord files to raw format
    ds = tf.data.TFRecordDataset(tfrecord_filenames)  # numf reads can be put here, but I don't think I/O is the bottleneck

    # raw format to imgs,tensors(coords kpts)
    ds = ds.map(TF_parser.read_tfrecord)

    # cache  ,caching is here before decompressing jpgs and label tensors (should be ~9GB) , (full dataset should be ~90, cache later if RAM aviable)
    if CACHE: ds = ds.cache()
    if SHUFFLE: ds = ds.shuffle(100)

    # Augmentation should be here, to operate on smaller tensors

    # imgs,tensors to label_tensors (46,46,17/38)
    ds = ds.map(dataset_functions.make_label_tensors)
    # imgs,label_tensors arrange for model outputs
    ds = ds.map(dataset_functions.place_training_labels)

    ds = ds.batch(BATCH_SIZE)
    ds = ds.repeat()
    if PREFETCH: ds = ds.prefetch(PREFETCH)
    return ds
dst=build_training_ds(tfrecord_files_train)

Retrieving TFrecords in local mode


In [8]:
tfrecord_files_train,tfrecord_files_val=dataset_builder.get_tfrecord_filenames()
print("Found the following TFrecords:\n","\n".join(tfrecord_files_train),"\n", "\n".join(tfrecord_files_val))

print("Building training dataset")
dst=dataset_builder.build_training_ds(tfrecord_files_train)
print("Training dataset shape:",dst)
print("Building validation dataset")
dsv=dataset_builder.build_validation_ds(tfrecord_files_val)
print("Validation dataset shape:",dsv)

Retrieving TFrecords in local mode
Found the following TFrecords:
 .\dataset\TFrecords\training-001.tfrecords
.\dataset\TFrecords\training-002.tfrecords 
 .\dataset\TFrecords\validation-001.tfrecords
Building training dataset
Training dataset shape: <PrefetchDataset shapes: ((None, 368, 368, 3), ((None, 46, 46, 35), (None, 46, 46, 35), (None, 46, 46, 35), (None, 46, 46, 35), (None, 46, 46, 19), (None, 46, 46, 19))), types: (tf.float32, (tf.float32, tf.float32, tf.float32, tf.float32, tf.float32, tf.float32))>
Building validation dataset
Validation dataset shape: <BatchDataset shapes: ((None, 368, 368, 3), ((None, 46, 46, 35), (None, 46, 46, 35), (None, 46, 46, 35), (None, 46, 46, 35), (None, 46, 46, 19), (None, 46, 46, 19))), types: (tf.float32, (tf.float32, tf.float32, tf.float32, tf.float32, tf.float32, tf.float32))>


In [9]:
#Test element
# dst_iter=iter(dst)
# sample_elem=next(dst_iter)
# print("Dataset shape:",dst) #this should match the model input, and output stages

In [10]:
#testing mask
# sample_elem=next(dst_iter)
# m=sample_elem[1][1][0,...,0]
# import matplotlib.pyplot as plt
# plt.imshow(m)
# plt.colorbar()

---
# Callbacks

In [11]:
fit_callbacks=[
    callbacks.checkpoints_callback
    ,callbacks.tensorboard_callback
    ,callbacks.learning_rate_scheduler_callback
    ,callbacks.print_lr_callback
    ,tf.keras.callbacks.TerminateOnNaN()
]


In [12]:
#run to clean tensorboard results
#!gsutil -m rm -r {TENSORBOARD_PATH}/*
#!gsutil -m rm -r {CHECKPOINTS_PATH}/*



---
# Model
## Load weights

In [13]:
checkpoint=None
starting_epoch=0

In [14]:
#checkpoint,starting_epoch=load_weights.checkpoints_prompt()


In [15]:
if INCLUDE_MASK:
    loss=loss_metrics.MaskedMeanSquaredError
    loss_name="Masked_MSE"
else:
    loss=tf.keras.losses.MeanSquaredError
    loss_name="Regular_MSE"

In [16]:
model_maker=ModelMaker() #must be outside scope to keep the graph clean
tf.keras.backend.clear_session() #to clean to backaend from the imported model
def define():
    train_model,test_model=model_maker.create_models()
    
    #this must match the model output order
    metrics=[
          [tf.keras.metrics.MeanAbsoluteError()]
         ,[tf.keras.metrics.MeanAbsoluteError()]
         ,[tf.keras.metrics.MeanAbsoluteError()]
         ,[tf.keras.metrics.MeanAbsoluteError()]
         ,[#tf.keras.metrics.Recall(),
                    tf.keras.metrics.MeanAbsoluteError()]    
         ,[#tf.keras.metrics.Recall(),
                    tf.keras.metrics.MeanAbsoluteError()]
        ]
    
    train_model.compile(
                    optimizer=tf.keras.optimizers.Adam(BASE_LEARNING_RATE)
                    ,loss=loss(name=loss_name)
                    ,metrics=metrics
                    #,sample_weight_mode="temporal"        
                   )
    return train_model,test_model

if TPU_MODE:
    with strategy.scope():
        train_model,test_model=define()
        if (checkpoint):
            train_model.load_weights(checkpoint)
else:
    train_model,test_model=define()
    if (checkpoint):
        train_model.load_weights(checkpoint)

---
Actually training

In [17]:
train_history=train_model.fit(
    dst
    ,epochs=TRAINING_EPOCHS
    ,steps_per_epoch=STEPS_PER_EPOCH
    ,validation_data=dsv
    ,callbacks=fit_callbacks
    ,initial_epoch=starting_epoch
)

Train for 28000 steps

Learning rate for epoch 0 is 0.00019999999494757503
Epoch 1/100
   49/28000 [..............................] - ETA: 120:04:05 - loss: 0.3311 - stage1paf_output_m_loss: 0.0379 - stage2paf_output_m_loss: 0.0372 - stage3paf_output_m_loss: 0.0373 - stage4paf_output_m_loss: 0.0371 - stage5heatmap_output_m_loss: 0.0905 - stage6heatmap_output_m_loss: 0.0911 - stage1paf_output_m_mean_absolute_error: 0.0483 - stage2paf_output_m_mean_absolute_error: 0.0484 - stage3paf_output_m_mean_absolute_error: 0.0485 - stage4paf_output_m_mean_absolute_error: 0.0472 - stage5heatmap_output_m_mean_absolute_error: 0.1422 - stage6heatmap_output_m_mean_absolute_error: 0.1439
Epoch 00001: saving model to .\tmp\checkpoints\\05Thu1219-1849\Checkpoint-E0001.ckpt
   49/28000 [..............................] - ETA: 120:47:19 - loss: 0.3311 - stage1paf_output_m_loss: 0.0379 - stage2paf_output_m_loss: 0.0372 - stage3paf_output_m_loss: 0.0373 - stage4paf_output_m_loss: 0.0371 - stage5heatmap_output_m

KeyboardInterrupt: 

In [ ]:
model_path=MODELS_PATH+datetime.datetime.now().strftime("%d(%a)%m%y-%H%M")
tf.keras.models.save_model(train_model,model_path)


In [ ]:
#v.show_pafs_kpts_img(img.numpy(),paf.numpy(),kpt.numpy(),1,1) #can be used to draw the tensor data